# Adding memory

This shows how to add memory to an arbitrary chain. Right now, you can use the memory classes but need to hook it up manually

In [11]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [12]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [13]:
memory = ConversationBufferMemory(return_messages=True)

In [14]:
memory.load_memory_variables({})

{'history': []}

In [15]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [16]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
response

"Hello Bob! I'm glad to meet you. How can I help you today? If you have any questions or need assistance, feel free to ask."

In [17]:
memory.save_context(inputs, {"output": response})

In [18]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im bob'),
  AIMessage(content="Hello Bob! I'm glad to meet you. How can I help you today? If you have any questions or need assistance, feel free to ask.")]}

In [19]:
inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
response

"Hello Bob! As an AI language model, I don't have real-time awareness or the ability to know the names of specific individuals. However, I can certainly help you with any questions or topics you'd like to discuss. Please let me know how I can assist you."